# Single Map, some other parameter

* Use papermill to inject district parameter
* Within notebook, have it look up some column within the district and inject that column info into header

In [1]:
import branca
import geopandas as gpd
import pandas as pd

# Alternatively, can import relevant ones
from shared_utils import map_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
E0428 20:21:19.566850027    3511 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0428 20:21:21.971944194    3511 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [ ]:
## parameters cell
district = 7

In [ ]:
GEOSPATIAL_DATA = ("https://caltrans-gis.dot.ca.gov/arcgis/rest/services/"
       "CHboundary/District_Tiger_Lines/FeatureServer/0/query?"
       "where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&"
       "inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&"
       "relationParam=&outFields=DISTRICT%2C+Region&returnGeometry=true&"
       "maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&gdbVersion=&"
       "historicMoment=&returnDistinctValues=false&returnIdsOnly=false&"
       "returnCountOnly=false&returnExtentOnly=false&orderByFields=&"
       "groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&"
       "multipatchOption=xyFootprint&resultOffset=&resultRecordCount=&"
       "returnTrueCurves=false&returnExceededLimitFeatures=false&quantizationParameters=&"
       "returnCentroid=false&sqlFormat=none&resultType=&featureEncoding=esriDefault&"
       "datumTransformation=&f=geojson"
      )

gdf = gpd.read_file(GEOSPATIAL_DATA)

# Create a y-col to map (that's not just District number)
gdf = gdf.assign(
    some_y_col = gdf.index+10
)

gdf = gdf[gdf.DISTRICT==district]

In [ ]:
%%capture_parameters
some_metric = gdf.some_y_col.unique().tolist()
district, some_metric

# District {district} ------ grabbed metric: {some_metric}

In [ ]:
popup_dict = {
    "DISTRICT": "Caltrans District",
    "Region": "Caltrans Region",
    "some_y_col": "Something Measured"
}

colorscale = (branca.colormap.step.Accent_07
              .scale(vmin=gdf.DISTRICT.min(), 
                     vmax=gdf.DISTRICT.max())
             )

m = map_utils.make_folium_choropleth_map(
    gdf,
    plot_col = "DISTRICT",
    popup_dict = popup_dict,
    tooltip_dict = popup_dict,
    colorscale = colorscale,
    fig_width = 500, fig_height = 800,
    zoom = map_utils.REGION_CENTROIDS["CA"]["zoom"],
    centroid = map_utils.REGION_CENTROIDS["CA"]["centroid"],
    title=f"District {district} Map"
)

m